# Check that all the properties are consistent with the Kinds definition

- a way to group properties, and check if they are the same. 

an algorithm similar to the one used in the get_kinds, putting a threshold which is low. For now for scalar quantities...
Use the get_kinds with a zero threshold? in this way you find the properties which are different... or better, the number of different kinds.

Basically the algorithm is inside the get_kinds, which if I provide kind_tags, try to compute kinds with thr=0 and see if the mapping is the same. 
If it is the same, I should also not change any property values, but this I think is ok as we have thr=0 (maybe add a 1e-10).

**Main problem**: the recursion which I encounter when I try to access the `get_kinds()` from the Kinds property validation step.

When this works, put the custom threshold in the input of a given property. 

**However, I do not know if I want really to always set the kinds... or not?**

I do not know, because the user then will see the property values change.

RULE: Can also be not defined. In that case, the plugin will use the get_kinds() method to have the list of kinds to use. and there will be a snipper of code which generate a new SDATA,
        like `new_structure = StructureData.generate_with_kinds(old_structure)` which use the get_kinds...



Tests:

get_kinds works well for properties equal and thr = 0.


In [1]:
from aiida import orm, load_profile
load_profile()

from aiida_atomistic.data.structure import StructureData

where the value of each defined property is defined under the corresponding dictionary, under the key `value`. 


### (1) not providing kinds

we then need to use the get_kinds

In [2]:
unit_cell = [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
atomic_positions = [[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]]
symbols = ["Li"]*2
mass = [6.941,6.941]
charge = [0,1]
kinds = ["Li"]*2

properties = {
    "cell":{"value":unit_cell},
    "pbc":{"value":[True,True,True]},
    "positions":{"value":atomic_positions,},
    "symbols":{"value":symbols},
    "mass":{"value":mass,},
    "charge":{"value":charge},
    #"kinds":{"value":kinds}
    }

structure = StructureData(
        properties=properties
        )

kinds = structure.get_kinds()

print("Kinds: ", kinds)

print("Using the get_kinds to generate a new structure...")

properties.update(kinds[1]) 
properties["kinds"] = {"value":kinds[0]}

new_structure = StructureData(properties=properties)
print("...done.")
print("Kinds: ", new_structure.properties.kinds.value)
for i in new_structure.properties.get_stored_properties():
    print(i, new_structure.properties.get_property_attribute(i)['value'])

Kinds:  (['Li0', 'Li1'], {'mass': {'value': [6.941, 6.941]}, 'charge': {'value': [0.0, 1.0]}})
Using the get_kinds to generate a new structure...
...done.
Kinds:  ['Li0', 'Li1']
cell [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
mass [6.941, 6.941]
positions [[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]]
kinds ['Li0', 'Li1']
pbc [True, True, True]
charge [0.0, 1.0]
symbols ['Li', 'Li']


### (2) providing kinds

#### (2.1) wrong kinds

In [3]:
unit_cell = [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
atomic_positions = [[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]]
symbols = ["Li"]*2
mass = [6.941,6.941]
charge = [0,1]
kinds = ["Li"]*2 # Wrong, for the charge we provided, which should give rise to two different kinds.

properties = {
    "cell":{"value":unit_cell},
    "pbc":{"value":[True,True,True]},
    "positions":{"value":atomic_positions,},
    "symbols":{"value":symbols},
    "mass":{"value":mass,},
    "charge":{"value":charge},
    "kinds":{"value":kinds}
    }

structure = StructureData(
        properties=properties
        )

for i in new_structure.properties.get_stored_properties():
    print(i, new_structure.properties.get_property_attribute(i)['value'])

ValueError: The kinds you provided in the `kind_tags` input are not correct, as properties values are not consistent with them. Please check that this is what you want.

#### (2.1) correct kinds

In [6]:
unit_cell = [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
atomic_positions = [[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]]
symbols = ["Li"]*2
mass = [6.941,6.941]
charge = [0,1]
kinds = ["Li","Li1"]

properties = {
    "cell":{"value":unit_cell},
    "pbc":{"value":[True,True,True]},
    "positions":{"value":atomic_positions,},
    "symbols":{"value":symbols},
    "mass":{"value":mass,},
    "charge":{"value":charge},
    "kinds":{"value":kinds}
    }

structure = StructureData(
        properties=properties
        )

for i in structure.properties.get_stored_properties():
    print(i, structure.properties.get_property_attribute(i)['value'])

cell [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
mass [6.941, 6.941]
positions [[0.0, 0.0, 0.0], [1.5, 1.5, 1.5]]
kinds ['Li', 'Li1']
pbc [True, True, True]
charge [0, 1]
symbols ['Li', 'Li']
